In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.font_manager as fm
import re

# 맑은 고딕 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False  # 음수 기호가 깨지는 것을 방지

### 한국환경공단_전기차 충전소 위치 및 운영정보 불러오기

In [2]:
# 전국 전기차 충전소의 운영기관, 주소, 타입, 충전 용량, ID 등 (근데 아이디가 없다?)
# 환경부에서 ID 공개하지 않겠다고 했다네요,  , 왜 내게 이런 시련을
# 2012 ~ 2024/07/08 업데이트

charger = pd.read_csv("한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20230531.csv",
                     encoding = "cp949")
charger.head()

C:\Users\User\AppData\Local\Temp\ipykernel_13088\2377944207.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  charger = pd.read_csv("한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20230531.csv",


,설치년도,시도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),운영기관(대),운영기관(소),급속충전량,충전기타입,이용자제한,위도경도
0,2017,서울특별시,강서구,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5372628,126.8383789"
1,2017,서울특별시,강서구,서울특별시 강서구 강서로5길 50,곰달래문화복지센터 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5289561,126.8490887"
2,2017,서울특별시,강서구,서울특별시 강서구 까치산로4길 22,볏골공원 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5419747,126.8444499"
3,2017,서울특별시,강서구,서울특별시 강서구 우장산로 114,화곡6-1 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5530457,126.8488169"
4,2017,서울특별시,관악구,서울특별시 관악구 관천로 98,신림동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.4874046,126.9235075"


In [3]:
# 시도 확인
print("charger 시도 확인")
print(charger['시도'].unique())

# 서울특별시로 지역 한정
s_charger = charger[charger['시도'] == "서울특별시"]

# 데이터 기간 확인
print("\n[s_charger 기간 확인]")
print(s_charger['설치년도'].value_counts().sort_index())

# 겹치는 주소 확인
print("\n[s_charger 주소 확인]")
s_charger['주소'].value_counts()

charger 시도 확인
['서울특별시' '제주특별자치도' '전라남도' '경상북도' '강원특별자치도' '부산광역시' '충청남도' '충청북도' '인천광역시'
 '경기도' '대전광역시' '경상남도' '전북특별자치도' '대구광역시' '울산광역시' '광주광역시' '세종특별자치시']

[s_charger 기간 확인]
2012        9
2013        4
2015       15
2016      134
2017     1508
2018     1546
2019     1661
2020     2474
2021     8176
2022    19291
2023    15641
2024     6629
Name: 설치년도, dtype: int64

[s_charger 주소 확인]


서울특별시 송파구 올림픽로 135                      291
서울특별시 송파구 송파대로 345                      257
서울특별시 서대문구 가재울미래로 2                     243
서울특별시 송파구 올림픽로 99                       231
서울특별시 강서구 공항대로 103                      223
                                       ... 
서울특별시 은평구 연서로50길 12-7                     1
서울특별시 용산구 두핍바위로37길 6-2                    1
서울특별시 강북구 삼양로79길 39-26, 402호(삼성월드빌2)      1
서울특별시 성동구 둘레15길 11                        1
서울특별시 성동구 서울숲길 41                         1
Name: 주소, Length: 7651, dtype: int64

In [4]:
# 2022년도 이전에 설치된 충전기만
s_charger = s_charger[s_charger['설치년도'] < 2022]

# 주소별, 기종별로 그룹핑 + 기종 별 개수 카운트
s_charger['충전기 개수(기종별)'] = s_charger.groupby(['주소','기종(대)'])['기종(대)'].transform('size')
# 중복되는 값 일단 삭제 처리
s_charger = s_charger.drop_duplicates(subset = ['주소', '기종(대)'])


s_charger.head()

,설치년도,시도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),운영기관(대),운영기관(소),급속충전량,충전기타입,이용자제한,위도경도,충전기 개수(기종별)
0,2017,서울특별시,강서구,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5372628,126.8383789",1
1,2017,서울특별시,강서구,서울특별시 강서구 강서로5길 50,곰달래문화복지센터 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5289561,126.8490887",1
2,2017,서울특별시,강서구,서울특별시 강서구 까치산로4길 22,볏골공원 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5419747,126.8444499",2
3,2017,서울특별시,강서구,서울특별시 강서구 우장산로 114,화곡6-1 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5530457,126.8488169",2
4,2017,서울특별시,관악구,서울특별시 관악구 관천로 98,신림동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.4874046,126.9235075",1


In [5]:
# 필요한 컬럼만 추출, 컬럼명 변경
b = s_charger.drop(columns = ['시도', '군구', '운영기관(소)', '설치년도'])
b = b.rename(columns = {'기종(대)' : '충전기구분'})


# 주소에서 상세 주소 부분(필요 없음, 나중에 merge할 때 방해됨) 삭제하는 함수
def remove_num_and_parentheses(address):
    if isinstance(address, str):  # 주소가 문자열일 때만 처리
        # 괄호 안의 내용과 숫자 제거
        address = re.sub(r'\(\d*\)', '', address)  # 괄호와 괄호 안의 내용 제거
        return address.strip()  # 양옆 공백 제거
    return address  # 문자열이 아닌 경우 그대로 반환

# 급속 충전기
fast_charger = b[b['충전기구분'] == '급속']

#필요 없는 주소 삭제 및 공백 제거
fast_charger['주소2'] = fast_charger['주소'].apply(remove_num_and_parentheses)
fast_charger['주소2'] = fast_charger['주소2'].str.replace(' ', '', regex=False)

# 완속 충전기
slow_charger = b[b['충전기구분'] == '완속']

#필요 없는 주소 삭제 및 공백 제거
slow_charger['주소2'] = slow_charger['주소'].apply(remove_num_and_parentheses)
slow_charger['주소2'] = slow_charger['주소2'].str.replace(' ', '', regex=False)

print(fast_charger.info())
print()
print()
print(slow_charger.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 846 entries, 0 to 361151
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   주소           846 non-null    object
 1   충전소명         846 non-null    object
 2   시설구분(대)      846 non-null    object
 3   시설구분(소)      846 non-null    object
 4   충전기구분        846 non-null    object
 5   기종(소)        846 non-null    object
 6   운영기관(대)      846 non-null    object
 7   급속충전량        162 non-null    object
 8   충전기타입        846 non-null    object
 9   이용자제한        846 non-null    object
 10  위도경도         844 non-null    object
 11  충전기 개수(기종별)  846 non-null    int64 
 12  주소2          846 non-null    object
dtypes: int64(1), object(12)
memory usage: 92.5+ KB
None


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3091 entries, 8535 to 322250
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   주소           3091 n

C:\Users\User\AppData\Local\Temp\ipykernel_13088\2618823927.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fast_charger['주소2'] = fast_charger['주소'].apply(remove_num_and_parentheses)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\2618823927.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fast_charger['주소2'] = fast_charger['주소2'].str.replace(' ', '', regex=False)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\2618823927.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

### 한국전력공사_전기차 충전소 충전량 불러오기

In [6]:
#한전 전기차 충전소 사용 현황 데이터
#오픈 api 과정은 1_확인하기

use_20211231 = pd.read_csv("한국전력공사_전기차 충전소 충전량_20211231.csv")
display(use_20211231.head())
print("\n[데이터 기간 확인]")
print(use_20211231['충전시작시각'].max())
print(use_20211231['충전시작시각'].min())

,본부,사업소,주소,충전기,충전기ID,충전기구분,충전량,충전분,충전소명,충전시간,충전시작시각,충전용량,충전종료시각,충전종료일자
0,강원,강원직할,강원도 춘천시 수풍골길 7,완속05,1296,완속,13.17,39,강원직할,2,2021-07-01,7.0,2021-07-01 22:15:42,2021-07-01
1,남서울,남서울직할,서울특별시 영등포구 여의도동 21,급속01,8858,급속,15.84,24,남서울직할,0,2021-07-01,50.0,2021-07-01 14:54:51,2021-07-01
2,남서울,남서울직할,서울특별시 영등포구 여의도동 21,급속01,8858,급속,11.70,43,남서울직할,0,2021-07-01,50.0,2021-07-01 17:36:36,2021-07-01
3,강원,강원직할,강원도 춘천시 수풍골길 7,완속02,1293,완속,5.40,23,강원직할,1,2021-07-01,7.0,2021-07-01 12:22:29,2021-07-01
4,강원,강원직할,강원도 춘천시 수풍골길 7,완속02,1293,완속,5.53,11,강원직할,2,2021-07-01,7.0,2021-07-01 18:58:12,2021-07-01



[데이터 기간 확인]
2021-10-17
2021-07-01


In [7]:
#한전 전기차 충전소 사용 현황 데이터
#오픈 api 과정은 1_확인하기

use_20210630 = pd.read_csv("한국전력공사_전기차 충전소 충전량_20210630.csv")
print("\n[데이터 기간 확인]")
print(use_20210630['충전시작시각'].max())
print(use_20210630['충전시작시각'].min())


[데이터 기간 확인]
2021-05-15
2021-01-01


=> 충전량 데이터: 2021 01 01 ~ 2021 10 17
* 6월, 11월, 12월 미비

In [8]:
#20210101 ~ 20211131 데이터 병합

use = pd.concat([use_20210630, use_20211231], axis = 0, ignore_index = True)
use.head()

,본부,사업소,주소,충전기,충전기ID,충전기구분,충전량,충전분,충전소명,충전시간,충전시작시각,충전용량,충전종료시각,충전종료일자
0,인천,김포,경기도 김포시 전원로 44,급속01,1547,급속,28.92,49,운양동 전원마을월드2단지,0,2021-01-01,50.0,2021-01-01 22:23:44,2021-01-01
1,인천,시흥,경기도 시흥시 동서로 1068 (조남동),완속03,3143,완속,12.51,55,LH퍼스트리움 아파트,3,2021-01-01,7.0,2021-01-01 22:24:38,2021-01-01
2,남서울,강남,서울특별시 강남구 자곡동 자곡로3길 21,급속01,6452,급속,26.37,38,LH강남힐스테이트,0,2021-01-01,50.0,2021-01-01 22:24:33,2021-01-01
3,강원,강릉,강원도 평창군 대관령면 솔봉로 325,급속02,6681,급속,39.12,53,알펜시아리조트,0,2021-01-01,50.0,2021-01-01 17:05:27,2021-01-01
4,대전세종충남,서대전,대전광역시 서구 둔산대로 181,급속02,6175,급속,30.42,42,대전시립연정국악원 주차장,0,2021-01-01,50.0,2021-01-01 23:57:44,2021-01-01


In [9]:
# 주소가 Na인 데이터 추출
display(use[use['주소'].isna()])

# 주소가 Na인 데이터의 본부 추출
print("주소가 Na인 데이터의 본부")
print(use[use['주소'].isna()]['본부'].unique())

#환경부 소관의 서울 소재 충전기가 파악되지 않음

,본부,사업소,주소,충전기,충전기ID,충전기구분,충전량,충전분,충전소명,충전시간,충전시작시각,충전용량,충전종료시각,충전종료일자
4841,환경부,환경부,NaN,급속02,11004750,급속,18.32,40,문화재청 왕릉유적본부 서부지구관리소(김포장릉),0,2021-01-01,NaN,2021-01-01 08:39:12,2021-01-01
5713,환경부,환경부,NaN,급속01,11003298,급속,31.77,40,망향휴게소(부산방향),0,2021-01-01,NaN,2021-01-01 01:57:45,2021-01-01
25722,환경부,환경부,NaN,급속02,11003310,급속,6.73,15,달서별빛캠프 캠핑장,0,2021-01-04,NaN,2021-01-04 13:56:33,2021-01-04
29714,환경부,환경부,NaN,급속01,11002280,급속,9.99,43,롯데 VIC마켓 금천점,0,2021-01-05,NaN,2021-01-05 22:33:40,2021-01-05
29837,환경부,환경부,NaN,급속01,11003298,급속,20.57,40,망향휴게소(부산방향),0,2021-01-05,NaN,2021-01-05 21:56:08,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083392,환경부,환경부,NaN,급속01,11004040,급속,39.94,43,(재)문화엑스포,0,2021-10-16,NaN,2021-10-16 16:47:58,2021-10-16
2083873,환경부,환경부,NaN,급속01,11003723,급속,20.85,33,이마트 은평점,0,2021-10-16,NaN,2021-10-16 18:13:18,2021-10-16
2084514,환경부,환경부,NaN,급속02,11004041,급속,2.45,9,(재)문화엑스포,0,2021-10-16,NaN,2021-10-16 12:13:23,2021-10-16
2088173,환경부,환경부,NaN,급속02,11007052,급속,24.04,30,한려해상국립공원 연화 주차장,0,2021-10-17,NaN,2021-10-17 12:39:42,2021-10-17


주소가 Na인 데이터의 본부
['환경부' '제주']


In [10]:
# 환경부 서울시 소재 충전기 주소 딕셔너리 정의
address = {
    "가락시장" : "서울특별시 송파구 양재대로 932",
    "롯데 VIC마켓 금천점" : "서울특별시 금천구 두산로 71",
    "이마트 은평점" : "서울특별시 은평구 은평로 111",
    "강서구민올림픽체육센터" : "서울특별시 강서구 화곡로65길 62",

}

# 환경부 서울시 소재 충전기 주소 입력
use['주소'] = use['충전소명'].map(address).fillna(use['주소'])

# 주소값이 이상한 애들 돌려놓기
use.loc[use['충전소명'] == "방배롯데캐슬아르떼", '주소'] = "서울특별시 서초구 방배천로18길 11"
use.loc[use['충전소명'] == "양재리본타워2단지아파트", '주소'] = "서울 서초구 매헌로16길 40"


In [11]:
# 서울특별시 데이터만 추출
s_use = use[use['주소'].str.contains(r'^서울', na = False, regex = True)]

# 충전시간 분으로 통합
s_use['충전시간(분)'] = s_use['충전시간']*60 + s_use['충전분']

s_use.drop(columns = ['충전기', '충전분', '충전시간', '충전용량', '충전시작시각', '충전종료일자'],
          inplace = True)

C:\Users\User\AppData\Local\Temp\ipykernel_13088\2412467725.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_use['충전시간(분)'] = s_use['충전시간']*60 + s_use['충전분']
C:\Users\User\AppData\Local\Temp\ipykernel_13088\2412467725.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_use.drop(columns = ['충전기', '충전분', '충전시간', '충전용량', '충전시작시각', '충전종료일자'],


In [12]:
# 필요한 컬럼만 추출
a = s_use.groupby(['본부', '주소', '충전기구분', '충전소명'], as_index = False).sum()

#급속충전기 사용량
fast_use = a[a['충전기구분'] == '급속']

#필요 없는 주소 삭제 및 공백 제거
fast_use['주소2'] = fast_use['주소'].apply(remove_num_and_parentheses)
fast_use['주소2'] = fast_use['주소2'].str.replace(' ', '', regex=False)

#왼속충전기 사용량
slow_use = a[a['충전기구분'] == '완속']

#필요 없는 주소 삭제 및 공백 제거
slow_use['주소2'] = slow_use['주소'].apply(remove_num_and_parentheses)
slow_use['주소2'] = slow_use['주소2'].str.replace(' ', '', regex=False)


C:\Users\User\AppData\Local\Temp\ipykernel_13088\1550503869.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fast_use['주소2'] = fast_use['주소'].apply(remove_num_and_parentheses)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\1550503869.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fast_use['주소2'] = fast_use['주소2'].str.replace(' ', '', regex=False)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\1550503869.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

### 두 데이터 병합 (1) - 급속

In [13]:
display(fast_use.head())
display(fast_charger.head())

,본부,주소,충전기구분,충전소명,충전기ID,충전량,충전시간(분),주소2
1,남서울,서울특별시 강남구 논현동 276-1,급속,아크로힐스논현,141894,270.74,902,서울특별시강남구논현동276-1
3,남서울,서울특별시 강남구 대치동 도곡로93길 12,급속,래미안대치하이스턴 아파트,116800,1441.87,1948,서울특별시강남구대치동도곡로93길12
5,남서울,서울특별시 강남구 대치동 삼성로51길 37,급속,래미안대치팰리스,823132,9575.65,27172,서울특별시강남구대치동삼성로51길37
6,남서울,서울특별시 강남구 대치동 삼성로64길 5,급속,대치현대 아파트,127688,1403.22,2899,서울특별시강남구대치동삼성로64길5
8,남서울,서울특별시 강남구 도곡동 선릉로 221,급속,도곡렉슬 아파트,2102927,21881.97,76844,서울특별시강남구도곡동선릉로221


,주소,충전소명,시설구분(대),시설구분(소),충전기구분,기종(소),운영기관(대),급속충전량,충전기타입,이용자제한,위도경도,충전기 개수(기종별),주소2
0,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5372628,126.8383789",1,서울특별시강서구가로공원로189
1,서울특별시 강서구 강서로5길 50,곰달래문화복지센터 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5289561,126.8490887",1,서울특별시강서구강서로5길50
2,서울특별시 강서구 까치산로4길 22,볏골공원 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5419747,126.8444499",2,서울특별시강서구까치산로4길22
3,서울특별시 강서구 우장산로 114,화곡6-1 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5530457,126.8488169",2,서울특별시강서구우장산로114
4,서울특별시 관악구 관천로 98,신림동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.4874046,126.9235075",1,서울특별시관악구관천로98


In [14]:
### 1. 주소2가 같은 애들끼리 merge

# 첫 번째 기준으로 merge
fast_df = pd.merge(fast_use, fast_charger, on=['주소2', '충전기구분'], how = 'right')

# 충전소 명이 다른 것 중 틀린 정보 수정
fast_df.loc[((fast_df['충전소명_x'] == "강남한양수자인") & 
         (fast_df['충전소명_y'] == "리센츠 아파트")), '충전소명_y'] = np.nan

fast_df.loc[((fast_df['충전소명_x'] == "한국예술종합학교(종로)") & 
         (fast_df['충전소명_y'] == "한국예술종합학교(종로)")), '충전기ID'] = np.nan

fast_df.loc[((fast_df['충전소명_x'] == "강남데시앙파크") & 
         (fast_df['충전소명_y'] == "세곡리엔파크5단지")), '충전기ID'] = np.nan

fast_df.loc[((fast_df['충전소명_x'] == "이마트 양재점") & 
         (fast_df['충전소명_y'] == "양재하이브랜드")), '충전기ID'] = np.nan

fast_df.loc[((fast_df['충전소명_x'] == "답십리래미안위브2블럭") & 
         (fast_df['충전소명_y'] == "답십리래미안위브1블럭")), '충전소명_y'] = '답십리래미안위브2블럭'

fast_df.loc[((fast_df['충전소명_x'] == "롯데백화점 본점") & 
         (fast_df['충전소명_y'] == "롯데호텔 서울")), '충전기ID'] = np.nan

# 충전소명Y 삭제, 충전소명 컬럼명 재정비
fast_df.drop(columns = ['충전소명_x', '주소_x'], inplace = True)
fast_df = fast_df.rename(columns = {'충전소명_y' : '충전소명', '주소_y' : '주소'})

# 매치되지 않은 값 추출하기
unmerge = fast_df[fast_df['충전기ID'].isna()]
unmerge.drop(columns = ['본부', '충전기ID', '충전량', '충전시간(분)'], inplace = True)

# 매치되지 않은 값 일단 삭제
fast_df = fast_df.dropna(subset = ['충전기ID'])



### 2.충전소명이 같은 애들끼리 merge
# (매치되지 않은 값들 주소2가 아닌 충전소 명으로 merge해보기)
unmerge = pd.merge(fast_use, unmerge, on=['충전소명', '충전기구분'], how = 'right')

# 제대로 매치된 값만 c로 저장해서 fast_df에 이어붙이기
c = unmerge[(unmerge['충전기ID'].notna())]

## 주소2x 삭제, 충전소명 컬럼명 재정비
c.drop(columns = ['주소2_y', '주소_y'], inplace = True)
c = c.rename(columns = {'주소2_x' : '주소2', '주소_x' : '주소'})

fast_df = pd.concat([fast_df, c], axis = 0, ignore_index = True)
fast_df.head()


C:\Users\User\AppData\Local\Temp\ipykernel_13088\23966275.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmerge.drop(columns = ['본부', '충전기ID', '충전량', '충전시간(분)'], inplace = True)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\23966275.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c.drop(columns = ['주소2_y', '주소_y'], inplace = True)


,본부,충전기구분,충전기ID,충전량,충전시간(분),주소2,주소,충전소명,시설구분(대),시설구분(소),기종(소),운영기관(대),급속충전량,충전기타입,이용자제한,위도경도,충전기 개수(기종별)
0,환경부,급속,132006072.0,245.35,370.0,서울특별시강서구가로공원로189,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5372628,126.8383789",1
1,환경부,급속,55002535.0,26.75,136.0,서울특별시강서구강서로5길50,서울특별시 강서구 강서로5길 50,곰달래문화복지센터 공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5289561,126.8490887",1
2,환경부,급속,11000508.0,2.65,6.0,서울특별시강서구까치산로4길22,서울특별시 강서구 까치산로4길 22,볏골공원 지하공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5419747,126.8444499",2
3,환경부,급속,33001527.0,27.95,51.0,서울특별시강서구우장산로114,서울특별시 강서구 우장산로 114,화곡6-1 공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5530457,126.8488169",2
4,환경부,급속,121005621.0,143.46,257.0,서울특별시관악구관천로98,서울특별시 관악구 관천로 98,신림동 제1공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.4874046,126.9235075",1


In [15]:
### 그래도 매치되지 않은 값 있는 지 확인
### (주소 형식 차이로 매치되지 않은 값 확인)
missing = fast_use[ ~fast_use['충전기ID'].isin(fast_df['충전기ID'])]

#도로명만 추출하는 함수 정의
def extract_road_name(address):
    match = re.findall(r'[가-힣]+(?:로|길|가)', address)  
    if match:
        return match
    return None

def remove(address):
    if isinstance(address, str):  # 주소가 문자열일 때만 처리
        # 대괄호와 작은 따옴표를 모두 제거
        address = re.sub(r'[\[\]\'"]', '', address)
        return address.strip()  # 양옆 공백 제거
    return address  # 문자열이 아닌 경우 그대로 반환

missing['도로명'] = missing['주소'].apply(extract_road_name)
missing['도로명'] = missing['도로명'].apply(lambda x: str(x) if isinstance(x, list) else x)

missing['도로명'] = missing['도로명'].apply(remove)
missing['번호'] = missing['주소'].str.extract(r'(\d+)', expand=False)

fast_charger['도로명'] = fast_charger['주소'].apply(extract_road_name)
fast_charger['도로명'] = fast_charger['도로명'].apply(lambda x: str(x) if isinstance(x, list) else x)

fast_charger['도로명'] = fast_charger['도로명'].apply(remove)
fast_charger['번호'] = fast_charger['주소'].str.extract(r'(\d+)', expand=False)

# 도로명 + 주소로 merge 시도
d = pd.merge(missing, fast_charger, on =['도로명', '번호'], how = 'left')

# 매치된 값 확인
pd.set_option('display.max_rows', None)
d[d['주소_y'].notna()][['주소_x', '주소_y', '충전소명_x', '충전소명_y']]

C:\Users\User\AppData\Local\Temp\ipykernel_13088\738742925.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing['도로명'] = missing['주소'].apply(extract_road_name)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\738742925.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing['도로명'] = missing['도로명'].apply(lambda x: str(x) if isinstance(x, list) else x)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\738742925.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,주소_x,주소_y,충전소명_x,충전소명_y
2,서울특별시 강남구 세곡동 헌릉로 590길10,서울특별시 강남구 세곡동 헌릉로590길 63,세곡리엔파크2단지 아파트,세곡리엔파크5단지
3,서울특별시 강남구 세곡동 헌릉로 590길10,서울특별시 강남구 세곡동 헌릉로590길 11,세곡리엔파크2단지 아파트,세곡리엔파크3단지
4,서울특별시 강남구 세곡동 헌릉로590길 63,서울특별시 강남구 세곡동 헌릉로590길 63,강남데시앙파크,세곡리엔파크5단지
5,서울특별시 강남구 세곡동 헌릉로590길 63,서울특별시 강남구 세곡동 헌릉로590길 11,강남데시앙파크,세곡리엔파크3단지
10,"서울특별시 강서구 화곡로 308 (화곡동, 서울강서경찰서)",서울특별시 강서구 화곡로 308,강서경찰서 민원인주차장,서울강서경찰서
14,"서울특별시 구로구 벚꽃로 370 (가리봉동, 하늘공원주차장)",서울특별시 구로구 벚꽃로 370,하늘공원 공영주차장,하늘공원주차장
15,"서울특별시 구로구 벚꽃로 370 (가리봉동, 하늘공원주차장)",서울특별시 구로구 벚꽃로 370,하늘공원 공영주차장,하늘공원
19,서울특별시 동작구 상도동 상도로 346-2,서울특별시 동작구 상도로 346-2,힐스테이트상도프레스티지,힐스테이트 상도 프레스티지
22,서울특별시 송파구 거여동 양산로4길 8,서울특별시 송파구 거여동 양산로4길 16,거여4단지 아파트,거여5단지 아파트
26,서울특별시 송파구 충민로 4길 19,서울특별시 송파구 장지동 충민로4길 5,송파파인타운7단지,송파파인타운5단지


In [16]:
# 형식이 달라 매치되지 않았던 값들 추출만 필터링
missing = d[(d['주소_x'] == "서울특별시 강남구 세곡동 헌릉로 590길10")  |
  (d['주소_x'] == "서울특별시 강서구 화곡로 308 (화곡동, 서울강서경찰서)") | 
  (d['주소_x'] == "서울특별시 구로구 벚꽃로 370 (가리봉동, 하늘공원주차장)") |
  (d['주소_x'] == "서울특별시 동작구 상도동 상도로 346-2") |
  (d['주소_x'] == "서울특별시 송파구 거여동 양산로4길 8") |
  (d['주소_x'] == "서울특별시 송파구 충민로 4길 19") |
  (d['주소_x'] == "서울특별시 양천구 월정로 46 (신월동)") |
  (d['주소_x'] == "서울특별시 노원구 노원로 75 (공릉동, 한국원자력의학원)") |
  (d['주소_x'] == "서울특별시 마포구 성산로 128 (성산동, 마포중앙도서관)") |
  (d['주소_x'] == "서울특별시 성북구 안암로 145") |
  (d['주소_x'] == "서울특별시 종로구 삼청로 30 (소격동)") |
  (d['주소_x'] == "서울특별시 종로구 종로5길 58 (수송동, 석탄회관빌딩)") |
  (d['주소_x'] == "서울특별시 마포구 마포대로1길 9") |
  (d['주소_x'] == "서울특별시 종로구 세종대로 189") | 
  (d['주소_x'] == "서울특별시 중구 을지로 30") 
 ]

# 중복되는 값 삭제
missing = missing.drop_duplicates(subset = ['충전기ID', '충전량', '충전시간(분)'])

# 컬럼명 정리
missing = missing.rename(columns = {'주소_x' : '주소',
                                    '주소2_x' : '주소2',
                                  '충전기구분_x' : '충전기구분',
                                  '충전소명_x' : '충전소명'})
missing = missing[fast_df.columns]

# fast_df로 병합
fast_df = pd.concat([fast_df, missing], axis = 0, ignore_index = True)
display(fast_df.head())
display(fast_df.info())

,본부,충전기구분,충전기ID,충전량,충전시간(분),주소2,주소,충전소명,시설구분(대),시설구분(소),기종(소),운영기관(대),급속충전량,충전기타입,이용자제한,위도경도,충전기 개수(기종별)
0,환경부,급속,132006072.0,245.35,370.0,서울특별시강서구가로공원로189,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5372628,126.8383789",1.0
1,환경부,급속,55002535.0,26.75,136.0,서울특별시강서구강서로5길50,서울특별시 강서구 강서로5길 50,곰달래문화복지센터 공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5289561,126.8490887",1.0
2,환경부,급속,11000508.0,2.65,6.0,서울특별시강서구까치산로4길22,서울특별시 강서구 까치산로4길 22,볏골공원 지하공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5419747,126.8444499",2.0
3,환경부,급속,33001527.0,27.95,51.0,서울특별시강서구우장산로114,서울특별시 강서구 우장산로 114,화곡6-1 공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5530457,126.8488169",2.0
4,환경부,급속,121005621.0,143.46,257.0,서울특별시관악구관천로98,서울특별시 관악구 관천로 98,신림동 제1공영주차장,주차시설,공영주차장,급속(50kW),환경부,급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.4874046,126.9235075",1.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   본부           533 non-null    object 
 1   충전기구분        533 non-null    object 
 2   충전기ID        533 non-null    float64
 3   충전량          533 non-null    float64
 4   충전시간(분)      533 non-null    float64
 5   주소2          533 non-null    object 
 6   주소           533 non-null    object 
 7   충전소명         532 non-null    object 
 8   시설구분(대)      533 non-null    object 
 9   시설구분(소)      533 non-null    object 
 10  기종(소)        533 non-null    object 
 11  운영기관(대)      533 non-null    object 
 12  급속충전량        132 non-null    object 
 13  충전기타입        533 non-null    object 
 14  이용자제한        533 non-null    object 
 15  위도경도         533 non-null    object 
 16  충전기 개수(기종별)  533 non-null    float64
dtypes: float64(4), object(13)
memory usage: 70.9+ KB


None

### 두 데이터 병합 (2) - 완속

In [17]:
### 1. 주소2 또는 충전소명이 같은 애들끼리 merge

# 첫 번째 기준으로 merge
slow_df = pd.merge(slow_use, slow_charger, on=['주소2', '충전기구분'], how = 'right')

# 충전소 명이 다른 것 중 틀린 정보 수정
slow_df.loc[((slow_df['충전소명_x'] == "강남데시앙파크") & 
         (slow_df['충전소명_y'] == "세곡리엔파크5단지")), '충전기ID'] = np.nan

slow_df.loc[((slow_df['충전소명_x'] == "무악현대 아파트") & 
         (slow_df['충전소명_y'] == "인왕산2차아이파크")), '충전기ID'] = np.nan

slow_df.loc[((slow_df['충전소명_x'] == "공덕파크자이2단지") & 
         (slow_df['충전소명_y'] == "공덕파크자이1단지")), '충전기ID'] = np.nan

slow_df.loc[((slow_df['충전소명_x'] == "공덕파크자이1단지") & 
         (slow_df['충전소명_y'] == "공덕파크자이2단지")), '충전기ID'] = np.nan

slow_df.loc[((slow_df['충전소명_x'] == "용산한신 아파트") & 
         (slow_df['충전소명_y'] == "메세나폴리스 아파트")), '충전기ID'] = np.nan

slow_df.loc[((slow_df['충전소명_x'] == "강남한양수자인") & 
         (slow_df['충전소명_y'] == "GS강동자이 아파트")), '충전소명_y'] = "강남한양수자인"

slow_df.loc[((slow_df['충전소명_x'] == "강남한양수자인") & 
         (slow_df['충전소명_y'] == "고덕리엔파크1단지")), '충전기ID'] = np.nan


# 충전소명Y 삭제, 충전소명 컬럼명 재정비
slow_df.drop(columns = ['충전소명_x', '주소_y'], inplace = True)
slow_df = slow_df.rename(columns = {'충전소명_y' : '충전소명', '주소_x' : '주소'})

# 매치되지 않은 값 추출하기
unmerge = slow_df[slow_df['충전기ID'].isna()]
unmerge.drop(columns = ['본부', '충전기ID', '충전량', '충전시간(분)'], inplace = True)

# 매치되지 않은 값 일단 삭제
slow_df = slow_df.dropna(subset = ['충전기ID'])



### 1. 주소2 또는 충전소명이 같은 애들끼리 merge
# (매치되지 않은 값들 주소2가 아닌 충전소 명으로 merge해보기)
unmerge = pd.merge(slow_use, unmerge, on=['충전소명', '충전기구분'], how = 'right')

# 제대로 매치된 값만 c로 저장해서 slow_df에 이어붙이기
c = unmerge[(unmerge['충전기ID'].notna())]

## 주소2x 삭제, 충전소명 컬럼명 재정비
c.drop(columns = ['주소2_y', '주소_y'], inplace = True)
c = c.rename(columns = {'주소2_x' : '주소2', '주소_x' : '주소'})


slow_df = pd.concat([slow_df, c], axis = 0, ignore_index = True)
slow_df.head()


C:\Users\User\AppData\Local\Temp\ipykernel_13088\3501793704.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmerge.drop(columns = ['본부', '충전기ID', '충전량', '충전시간(분)'], inplace = True)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\3501793704.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c.drop(columns = ['주소2_y', '주소_y'], inplace = True)


,본부,주소,충전기구분,충전기ID,충전량,충전시간(분),주소2,충전소명,시설구분(대),시설구분(소),기종(소),운영기관(대),급속충전량,충전기타입,이용자제한,위도경도,충전기 개수(기종별)
0,서울,서울특별시 강북구 도봉로 242,완속,54301.0,1962.15,43438.0,서울특별시강북구도봉로242,강북성북지사(공용),공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용가능,"37.630583,127.0248869",3
1,서울,서울특별시 중랑구 동일로 862,완속,99034.0,979.33,38151.0,서울특별시중랑구동일로862,동대문중랑지사,공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용자제한,"37.60746854,127.0792448",7
2,서울,서울특별시 마포구 토정로37길 38,완속,111210.0,2659.71,66190.0,서울특별시마포구토정로37길38,마포용산지사,공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용자제한,"37.54205285,126.9466325",5
3,서울,서울특별시 중구 남대문로 92,완속,83933.0,1033.97,17189.0,서울특별시중구남대문로92,서울직할,공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용자제한,"37.56527705,126.9833704",6
4,서울,서울특별시 성동구 마장로27길 13,완속,262788.0,2255.85,46075.0,서울특별시성동구마장로27길13,광진성동지사,공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용자제한,"37.5677096,127.0340572",7


In [18]:
### 그래도 매치되지 않은 값 있는 지 확인
### (주소 형식 차이로 매치되지 않은 값 확인)
missing = slow_use[ ~slow_use['충전기ID'].isin(slow_df['충전기ID'])]

#도로명만 추출하는 함수 정의
def extract_road_name(address):
    match = re.findall(r'[가-힣]+(?:로|길|가)', address)  
    if match:
        return match
    return None

def remove(address):
    if isinstance(address, str):  # 주소가 문자열일 때만 처리
        # 대괄호와 작은 따옴표를 모두 제거
        address = re.sub(r'[\[\]\'"]', '', address)
        return address.strip()  # 양옆 공백 제거
    return address  # 문자열이 아닌 경우 그대로 반환

missing['도로명'] = missing['주소'].apply(extract_road_name)
missing['도로명'] = missing['도로명'].apply(lambda x: str(x) if isinstance(x, list) else x)

missing['도로명'] = missing['도로명'].apply(remove)
missing['번호'] = missing['주소'].str.extract(r'(\d+)', expand=False)

slow_charger['도로명'] = slow_charger['주소'].apply(extract_road_name)
slow_charger['도로명'] = slow_charger['도로명'].apply(lambda x: str(x) if isinstance(x, list) else x)

slow_charger['도로명'] = slow_charger['도로명'].apply(remove)
slow_charger['번호'] = slow_charger['주소'].str.extract(r'(\d+)', expand=False)

# 도로명 + 주소로 merge 시도
d = pd.merge(missing, slow_charger, on =['도로명', '번호'], how = 'left')

# 매치된 값 확인
pd.set_option('display.max_rows', None)
d[d['주소_y'].notna()][['주소_x', '주소_y', '충전소명_x', '충전소명_y']]

C:\Users\User\AppData\Local\Temp\ipykernel_13088\292841372.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing['도로명'] = missing['주소'].apply(extract_road_name)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\292841372.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing['도로명'] = missing['도로명'].apply(lambda x: str(x) if isinstance(x, list) else x)
C:\Users\User\AppData\Local\Temp\ipykernel_13088\292841372.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,주소_x,주소_y,충전소명_x,충전소명_y
0,서울특별시 강남구 도곡동 언주로30길 26,서울특별시 강남구 도곡동 언주로30길 56,타워팰리스G동,타워팰리스1차
1,서울특별시 강남구 도곡동 언주로30길 26,서울특별시 강남구 도곡동 언주로30길 57,타워팰리스G동,타워팰리스 2차
2,서울특별시 강남구 도곡동 언주로30길 26,"서울특별시 강남구 언주로30길 21 (도곡동, 아카데미스위트)",타워팰리스G동,아카데미스위트 아파트
3,서울특별시 강남구 도곡동 언주로30길 26,서울특별시 강남구 언주로30길 21,타워팰리스G동,이상훈
4,서울특별시 강남구 도곡동 언주로30길 26,서울특별시 강남구 언주로30길 56,타워팰리스G동,타워팰리스1차입주자대표회의
5,서울특별시 강남구 도곡동 언주로30길 26,서울특별시 강남구 언주로30길 13,타워팰리스G동,대림아크로빌
6,서울특별시 강남구 도곡동 언주로30길 26,서울특별시 강남구 언주로30길 57,타워팰리스G동,서울강남타워팰리스E동
8,서울특별시 강남구 세곡동 헌릉로590길 11,서울특별시 강남구 세곡동 헌릉로590길 63,세곡리엔파크3단지,세곡리엔파크5단지
9,서울특별시 강남구 세곡동 헌릉로590길 11,서울특별시 강남구 헌릉로590길 63,세곡리엔파크3단지,서울시 강남구 강남데시앙파크
10,서울특별시 강남구 세곡동 헌릉로590길 11,서울특별시 강남구 헌릉로590길 88,세곡리엔파크3단지,세곡지구4단지


In [19]:
# 형식이 달라 매치되지 않았던 값들 추출만 필터링
missing = d[(d['충전소명_y'] == "서울강남타워팰리스E동")  |
            (d['충전소명_y'] == "세곡리엔파크5단지") | 
            (d['충전소명_y'] == "강남데시앙포레 입주자대표회의") | 
            (d['충전소명_y'] == "마곡수명산파크6단지아파트공동주택대표회의") | 
            (d['충전소명_y'] == "서울노원건영3차") |
            (d['충전소명_y'] == "인왕산2차아이파크") |
            (d['주소_x'] == "서울특별시 동작구 상도동 상도로 346-2") |
            (d['주소_x'] == "서울특별시 송파구 송파대로 8길 17") |
            (d['주소_x'] == "서울특별시 송파구 충민로 4길 19") |
            (d['주소_x'] == "서울특별시 강북구 강북구 도봉로 242") 
           ]


# 중복되는 값 삭제
missing = missing.drop_duplicates(subset = ['충전기ID', '충전량', '충전시간(분)'])

# 컬럼명 정리
missing = missing.rename(columns = {'주소_x' : '주소',
                                    '주소2_x' : '주소2',
                                  '충전기구분_x' : '충전기구분',
                                  '충전소명_x' : '충전소명'})
missing = missing[slow_df.columns]

# slow_df로 병합
slow_df = pd.concat([slow_df, missing], axis = 0, ignore_index = True)
display(slow_df.head())
display(slow_df.info())

,본부,주소,충전기구분,충전기ID,충전량,충전시간(분),주소2,충전소명,시설구분(대),시설구분(소),기종(소),운영기관(대),급속충전량,충전기타입,이용자제한,위도경도,충전기 개수(기종별)
0,서울,서울특별시 강북구 도봉로 242,완속,54301.0,1962.15,43438.0,서울특별시강북구도봉로242,강북성북지사(공용),공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용가능,"37.630583,127.0248869",3.0
1,서울,서울특별시 중랑구 동일로 862,완속,99034.0,979.33,38151.0,서울특별시중랑구동일로862,동대문중랑지사,공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용자제한,"37.60746854,127.0792448",7.0
2,서울,서울특별시 마포구 토정로37길 38,완속,111210.0,2659.71,66190.0,서울특별시마포구토정로37길38,마포용산지사,공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용자제한,"37.54205285,126.9466325",5.0
3,서울,서울특별시 중구 남대문로 92,완속,83933.0,1033.97,17189.0,서울특별시중구남대문로92,서울직할,공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용자제한,"37.56527705,126.9833704",6.0
4,서울,서울특별시 성동구 마장로27길 13,완속,262788.0,2255.85,46075.0,서울특별시성동구마장로27길13,광진성동지사,공공시설,공공기관,AC완속,타기관,NaN,AC완속,이용자제한,"37.5677096,127.0340572",7.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   본부           436 non-null    object 
 1   주소           436 non-null    object 
 2   충전기구분        436 non-null    object 
 3   충전기ID        436 non-null    float64
 4   충전량          436 non-null    float64
 5   충전시간(분)      436 non-null    float64
 6   주소2          436 non-null    object 
 7   충전소명         436 non-null    object 
 8   시설구분(대)      436 non-null    object 
 9   시설구분(소)      436 non-null    object 
 10  기종(소)        436 non-null    object 
 11  운영기관(대)      436 non-null    object 
 12  급속충전량        0 non-null      object 
 13  충전기타입        436 non-null    object 
 14  이용자제한        436 non-null    object 
 15  위도경도         436 non-null    object 
 16  충전기 개수(기종별)  436 non-null    float64
dtypes: float64(4), object(13)
memory usage: 58.0+ KB


None

### 병합된 데이터에 정보 추가

In [20]:
# 충전기 1대당 충전시간 계산
fast_df['한대당 충전시간(분)'] = fast_df['충전시간(분)'] / fast_df['충전기 개수(기종별)']
slow_df['한대당 충전시간(분)'] = slow_df['충전시간(분)'] / slow_df['충전기 개수(기종별)']

# 위도, 경도 분리
fast_df[['위도', '경도']] = fast_df['위도경도'].str.split(',', expand=True)
slow_df[['위도', '경도']] = slow_df['위도경도'].str.split(',', expand=True)

# 구 추출하는 함수 정의
def ku(address):
    if isinstance(address, str):  # 문자열인지 확인
        match = re.findall(r'[가-힣]+(?:구)', address)
        if match:
            return match[0]  # 첫 번째 매칭 값 반환
    return None  # 문자열이 아니면 None 반환

fast_df['구'] = fast_df['주소'].apply(ku)
slow_df['구'] = slow_df['주소'].apply(ku)

# 필요없는 데이터 삭제
fast_df.drop(columns = ['주소2', '위도경도'], inplace = True)
slow_df.drop(columns = ['주소2', '위도경도'], inplace = True)


In [21]:
fast_df.to_csv("급속충전기사용량.csv", index = False)
slow_df.to_csv("완속충전기사용량.csv", index = False)